In [2]:
# ==============================================================
# 📘 01_data_init_clean.ipynb
# Sehr clean, RAM-safe, ohne Streamlit/Warnings/Spam.
# ==============================================================
import sys
import warnings
from pathlib import Path

warnings.filterwarnings("ignore")  # global clean mode

# --------------------------------------------------------------
# 0) Bootstrap project root
# --------------------------------------------------------------
ROOT = Path().resolve()
PROJECT_ROOT = ROOT.parent if ROOT.name == "notebooks" else ROOT
sys.path.append(str(PROJECT_ROOT))

print(f"📦 Project root: {PROJECT_ROOT}")

# --------------------------------------------------------------
# 1) Imports
# --------------------------------------------------------------
import pandas as pd
import yaml
from src.data_loader import DataLoader, ensure_dir

# --------------------------------------------------------------
# 2) Paths
# --------------------------------------------------------------
DATA_DIR    = PROJECT_ROOT / "data"
CONFIG_PATH = PROJECT_ROOT / "configs" / "data" / "active.yaml"

ensure_dir(DATA_DIR)

print(f"📁 DATA DIR:      {DATA_DIR}")
print(f"⚙️ CONFIG PATH:   {CONFIG_PATH} (exists={CONFIG_PATH.exists()})")

# --------------------------------------------------------------
# 3) YAML - auto-repair local_dir
# --------------------------------------------------------------
if CONFIG_PATH.exists():
    with open(CONFIG_PATH, "r", encoding="utf-8") as f:
        cfg = yaml.safe_load(f) or {}

    cfg.setdefault("source", {})
    correct_path = str(DATA_DIR)

    if str(cfg["source"].get("local_dir")) != correct_path:
        cfg["source"]["local_dir"] = correct_path
        with open(CONFIG_PATH, "w", encoding="utf-8") as f:
            yaml.safe_dump(cfg, f, sort_keys=False)
        print("🛠️ YAML auto-fixed → source.local_dir updated.")
    else:
        print("✅ YAML path already correct.")
else:
    raise FileNotFoundError(f"Missing config: {CONFIG_PATH}")

# --------------------------------------------------------------
# 4) DataLoader
# --------------------------------------------------------------
loader = DataLoader(config_path=CONFIG_PATH)
print("✅ DataLoader loaded.")

# --------------------------------------------------------------
# 5) Show raw CSV files
# --------------------------------------------------------------
print("\n🧾 Raw files BEFORE preprocessing:")
for f in sorted(DATA_DIR.glob("*.csv")):
    print("   ", f.name)

# --------------------------------------------------------------
# 6) Run preprocessing (CSV → Parquet)
# --------------------------------------------------------------
print("\n🚀 Running preprocessing (CSV → Parquet)…")
loader.preprocess_csv_to_parquet()
print("✅ Preprocessing complete.")

# --------------------------------------------------------------
# 7) Show produced Parquet files
# --------------------------------------------------------------
print("\n📦 Generated Parquet files:")
parq = sorted(DATA_DIR.glob("*.parquet"))
parts = sorted(DATA_DIR.glob("train_part*.parquet"))
partitioned = sorted(DATA_DIR.glob("train/**/*.parquet"))

for p in parq + parts + partitioned:
    size_mb = p.stat().st_size / 1e6
    rel = str(p.relative_to(DATA_DIR))
    print(f"   {rel:<40} {size_mb:>7.1f} MB")

# --------------------------------------------------------------
# 8) Fast TRAIN summary (NO full load)
# --------------------------------------------------------------
def collect_train_paths():
    files = sorted(DATA_DIR.glob("train_part*.parquet"))
    if files:
        return files

    files = sorted(DATA_DIR.glob("train/**/*.parquet"))
    if files:
        return files

    mono = DATA_DIR / "train.parquet"
    if mono.exists():
        return [mono]

    return []

train_paths = collect_train_paths()

print(f"\n🔗 Found {len(train_paths)} train part(s).")

# --------------------------------------------------------------
# 9) Load 10-row preview only (RAM safe)
# --------------------------------------------------------------
print("\n📋 Preview (first 10 rows from first train part):")
df_preview = pd.read_parquet(train_paths[0], engine="pyarrow").head(10)
display(df_preview)

# --------------------------------------------------------------
# 10) Chunkwise duplicate check (RAM-safe)
# --------------------------------------------------------------
print("\n🧹 Duplicate-check across ALL train parts…")

dup_count = 0
key_cols = ["id", "date", "store_nbr"]

for p in train_paths:
    df = pd.read_parquet(p, columns=key_cols)
    dup_count += df.duplicated(subset=key_cols).sum()

if dup_count == 0:
    print("✅ No duplicate (id, date, store_nbr) keys found.")
else:
    print(f"❗ Duplicate combinations detected: {dup_count:,}")

# --------------------------------------------------------------
# 11) Date range (RAM safe)
# --------------------------------------------------------------
print("\n📅 Computing date range…")
min_date, max_date = None, None

for p in train_paths:
    df = pd.read_parquet(p, columns=["date"])
    mn = df["date"].min()
    mx = df["date"].max()

    if min_date is None or mn < min_date:
        min_date = mn
    if max_date is None or mx > max_date:
        max_date = mx

print(f"📅 Date range: {min_date} → {max_date}")

# --------------------------------------------------------------
# 12) Summary
# --------------------------------------------------------------
print("\n🎯 CLEAN DATA INIT DONE.")
print("Next: open 02_eda_overview_clean.ipynb")

📦 Project root: /Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting
📁 DATA DIR:      /Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting/data
⚙️ CONFIG PATH:   /Users/kiko/Desktop/github/Corporacion-Favorita-Grocery-Sales-Forecasting/configs/data/active.yaml (exists=True)
✅ YAML path already correct.
✅ DataLoader loaded.

🧾 Raw files BEFORE preprocessing:
    holidays_events.csv
    items.csv
    oil.csv
    sample_submission.csv
    stores.csv
    test.csv
    train.csv
    transactions.csv

🚀 Running preprocessing (CSV → Parquet)…

➡️ Processing test.csv …
🧩 Wrote test_part1.parquet (3,370,464 rows)
✅ Done: test processed in 2.3s
🧹 Global dedup: 3,370,464 → 3,370,464
💾 Rewrote deduplicated parts
🎯 Finalized test

➡️ Processing train.csv …
🧩 Wrote train_part1.parquet (26,000,000 rows)
🧩 Wrote train_part2.parquet (26,000,000 rows)
🧩 Wrote train_part3.parquet (26,000,000 rows)
🧩 Wrote train_part4.parquet (25,857,647 rows)
✅ Done: train process

,id,date,store_nbr,item_nbr,unit_sales,onpromotion
0,0,2013-01-01,25.0,103665.0,7.0,<NA>
1,1,2013-01-01,25.0,105574.0,1.0,<NA>
2,2,2013-01-01,25.0,105575.0,2.0,<NA>
3,3,2013-01-01,25.0,108079.0,1.0,<NA>
4,4,2013-01-01,25.0,108701.0,1.0,<NA>
5,5,2013-01-01,25.0,108786.0,3.0,<NA>
6,6,2013-01-01,25.0,108797.0,1.0,<NA>
7,7,2013-01-01,25.0,108952.0,1.0,<NA>
8,8,2013-01-01,25.0,111397.0,13.0,<NA>
9,9,2013-01-01,25.0,114790.0,3.0,<NA>



🧹 Duplicate-check across ALL train parts…
✅ No duplicate (id, date, store_nbr) keys found.

📅 Computing date range…
📅 Date range: 2013-01-01 00:00:00 → 2017-01-22 00:00:00

🎯 CLEAN DATA INIT DONE.
Next: open 02_eda_overview_clean.ipynb
